<a href="https://colab.research.google.com/github/Shilpa393/Sql_Codes/blob/main/Shilpa_Sen_3C_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Postgres Exercises
https://pgexercises.com/ <br>
https://pgexercises.com/gettingstarted.html

In [ ]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-02-29 12:57:33.389304+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install & Configure Postgres

In [ ]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
#!pip install psycopg2
import psycopg2
import pandas as pd

#Create Schema, Tables and Insert Data

In [ ]:
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"

ALTER ROLE


In [ ]:
# Download and execute SQL file to create tables, load data
#
#!wget https://pgexercises.com/dbfiles/clubdata.sql
!wget -q https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/clubdata.sql
#Run the batch file to create tables, load data
!psql -h localhost -U postgres -f clubdata.sql -d postgres -x -q


Password for user postgres: 
psql:clubdata.sql:4: ERROR:  database "exercises" already exists
psql:clubdata.sql:6: ERROR:  schema "cd" already exists
psql:clubdata.sql:41: ERROR:  relation "bookings" already exists
psql:clubdata.sql:56: ERROR:  relation "facilities" already exists
psql:clubdata.sql:73: ERROR:  relation "members" already exists
psql:clubdata.sql:4126: ERROR:  insert or update on table "bookings" violates foreign key constraint "fk_bookings_memid"
DETAIL:  Key (memid)=(1) is not present in table "members".
psql:clubdata.sql:4144: ERROR:  duplicate key value violates unique constraint "facilities_pk"
DETAIL:  Key (facid)=(0) already exists.
psql:clubdata.sql:4193: ERROR:  multiple primary keys for table "bookings" are not allowed
psql:clubdata.sql:4202: ERROR:  multiple primary keys for table "facilities" are not allowed
psql:clubdata.sql:4211: ERROR:  multiple primary keys for table "members" are not allowed
psql:clubdata.sql:4220: ERROR:  constraint "fk_bookings_facid" 

In [ ]:
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:exercises:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:exercises:postgres:pass


In [ ]:
#Note new database called exercises
!psql -h localhost -p 5432 -Upostgres -c "\l"

Password for user postgres: 
                                  List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    |   Access privileges   
-----------+----------+----------+-------------+-------------+-----------------------
 exercises | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 postgres  | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | 
 template0 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_US.UTF-8 | en_US.UTF-8 | =c/postgres          +
           |          |          |             |             | postgres=CTc/postgres
(4 rows)



In [ ]:
#Note three tables under schema cd
!psql -h localhost -p 5432 -Upostgres -d exercises -c "\dt+ cd.*"

                                      List of relations
 Schema |    Name    | Type  |  Owner   | Persistence | Access method |  Size  | Description 
--------+------------+-------+----------+-------------+---------------+--------+-------------
 cd     | bookings   | table | postgres | permanent   | heap          | 264 kB | 
 cd     | facilities | table | postgres | permanent   | heap          | 16 kB  | 
 cd     | members    | table | postgres | permanent   | heap          | 16 kB  | 
(3 rows)



## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [ ]:
# Local Colab server

conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [ ]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="exercises", user='postgres', password='pass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

##Check Data

In [ ]:
runSelect("select * from cd.members limit 3")

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,None,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,None,2012-07-02 12:08:23


In [ ]:
runSelect("select * from cd.facilities limit 3")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50


In [ ]:
runSelect("select * from cd.bookings limit 3")

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


# 1) Basic Query

## 1.1 Retrieve everything form a table

In [ ]:
# How can you retrieve all the information from the cd.facilities table?

In [ ]:
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15


## 1.2 Retrieve specific columns from a table

In [ ]:
# You want to print out a list of all of the facilities and their cost to members. How would you retrieve a list of only facility names and costs?

In [ ]:
runSelect("select name, membercost from cd.facilities")

name,membercost
Tennis Court 1,5
Tennis Court 2,5
Badminton Court,0
Table Tennis,0
Massage Room 1,35
Massage Room 2,35
Squash Court,3.5
Snooker Table,0
Pool Table,0


## 1.3 Control which rows are retrieved

In [ ]:
# 3.1 How can you produce a list of facilities that charge a fee to members?
runSelect("select name, membercost, guestcost, initialoutlay, monthlymaintenance from cd.facilities where membercost > 0")

name,membercost,guestcost,initialoutlay,monthlymaintenance
Tennis Court 1,5,25,10000,200
Tennis Court 2,5,25,8000,200
Massage Room 1,35,80,4000,3000
Massage Room 2,35,80,4000,3000
Squash Court,3.5,17.5,5000,80


## 1.4 Control which rows are retrieved - part 2

In [ ]:
# 4.1 How can you produce a list of facilities that charge a fee to members, and that fee is less than 1/50th of the monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
runSelect("select name, membercost, monthlymaintenance from cd.facilities where membercost < monthlymaintenance/50 and membercost > 0")

name,membercost,monthlymaintenance
Massage Room 1,35,3000
Massage Room 2,35,3000


## 1.5 Basic String Searches

In [ ]:
# 5.1 How can you produce a list of all facilities with the word 'Tennis' in their name?
runSelect("select name, membercost, guestcost, initialoutlay, monthlymaintenance from cd.facilities where name like '%Tennis%'")

name,membercost,guestcost,initialoutlay,monthlymaintenance
Tennis Court 1,5,25,10000,200
Tennis Court 2,5,25,8000,200
Table Tennis,0,5,320,10


## 1.6 Matching against multiple possible values


In [ ]:
# 6.1 How can you retrieve the details of facilities with ID 1 and 5? Try to do it without using the OR operator.
runSelect("select facid, name, membercost, guestcost, initialoutlay, monthlymaintenance from cd.facilities where facid in (1,5)")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5,25,8000,200
5,Massage Room 2,35,80,4000,3000


## 1.7 Classify results into buckets

In [ ]:
# 7.1 How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive' depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

In [ ]:
runSelect("select name, case when monthlymaintenance > 100 then 'expensive' else 'cheap' end cost from cd.facilities")

name,cost
Tennis Court 1,expensive
Tennis Court 2,expensive
Badminton Court,cheap
Table Tennis,cheap
Massage Room 1,expensive
Massage Room 2,expensive
Squash Court,cheap
Snooker Table,cheap
Pool Table,cheap


## 1.8 Working with dates

In [ ]:
# How can you produce a list of members who joined after the start of September 2012? Return the memid, surname, firstname, and joindate of the members in question.
runSelect("select memid, surname, firstname, joindate from cd.members where joindate > '2012-09-01' ")

memid,surname,firstname,joindate
24,Sarwin,Ramnaresh,2012-09-01 08:44:42
26,Jones,Douglas,2012-09-02 18:43:05
27,Rumney,Henrietta,2012-09-05 08:42:35
28,Farrell,David,2012-09-15 08:22:05
29,Worthington-Smyth,Henry,2012-09-17 12:27:15
30,Purview,Millicent,2012-09-18 19:04:01
33,Tupperware,Hyacinth,2012-09-18 19:32:05
35,Hunt,John,2012-09-19 11:32:45
36,Crumpet,Erica,2012-09-22 08:36:38
37,Smith,Darren,2012-09-26 18:08:45


## 1.9 Removing duplicates, and ordering results

In [ ]:
# How can you produce an ordered list of the first 10 surnames in the members table? The list must not contain duplicates.

In [ ]:
runSelect("select distinct(surname) from cd.members order by surname limit 10")

surname
Bader
Baker
Boothe
Butters
Coplin
Crumpet
Dare
Farrell
Genting
GUEST


## 1.10 Combining results from multiple queries

In [ ]:
# You, for some reason, want a combined list of all surnames and all facility names. Yes, this is a contrived example :-). Produce that list!
runSelect("select surname from cd.members union select name from cd.facilities")

surname
Hunt
Farrell
Tennis Court 2
Table Tennis
Dare
Rownam
GUEST
Badminton Court
Smith
Tupperware


## 1.11 Simple aggregation

In [ ]:
# You'd like to get the signup date of your last member. How can you retrieve this information?
runSelect("select max(joindate) Latest_Signup from cd.members")

latest_signup
2012-09-26 18:08:45


## 1.12 More aggregation

In [ ]:
# You'd like to get the first and last name of the last member(s) who signed up - not just the date. How can you do that?

In [ ]:
runSelect("select firstname, surname, joindate from cd.members where joindate = (select max(joindate) from cd.members)")

firstname,surname,joindate
Darren,Smith,2012-09-26 18:08:45


# Joins and Subqueries

## 2.1 Retrieve the start times of members' bookings

In [ ]:
# How can you produce a list of the start times for bookings by members named 'David Farrell'?
runSelect("select starttime from cd.bookings, cd.members where cd.bookings.memid = cd.members.memid and cd.members.firstname = 'David' and cd.members.surname = 'Farrell'")

starttime
2012-09-18 09:00:00
2012-09-18 17:30:00
2012-09-18 13:30:00
2012-09-18 20:00:00
2012-09-19 09:30:00
2012-09-19 15:00:00
2012-09-19 12:00:00
2012-09-20 15:30:00
2012-09-20 11:30:00
2012-09-20 14:00:00


## 2.2 Work out the start times of bookings for tennis courts

In [ ]:
# How can you produce a list of the start times for bookings for tennis courts, for the date '2012-09-21'? Return a list of start time and facility name pairings, ordered by the time.
runSelect("select name, starttime as start from cd.bookings INNER JOIN cd.facilities ON cd.facilities.facid = cd.bookings.facid where name in ('Tennis Court 2','Tennis Court 1') and starttime >= '2012-09-21' and starttime < '2012-09-22' order by starttime")

name,start
Tennis Court 1,2012-09-21 08:00:00
Tennis Court 2,2012-09-21 08:00:00
Tennis Court 1,2012-09-21 09:30:00
Tennis Court 2,2012-09-21 10:00:00
Tennis Court 2,2012-09-21 11:30:00
Tennis Court 1,2012-09-21 12:00:00
Tennis Court 1,2012-09-21 13:30:00
Tennis Court 2,2012-09-21 14:00:00
Tennis Court 1,2012-09-21 15:30:00
Tennis Court 2,2012-09-21 16:00:00


## 2.3 Produce a list of all members who have recommended another member


In [ ]:
# How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).
runSelect("select distinct mem2.firstname as firstname, mem2.surname as surname from cd.members mem1 INNER JOIN cd.members mem2 on mem1.memid = mem2.recommendedby order by surname, firstname")

firstname,surname
Florence,Bader
Anne,Baker
Timothy,Baker
Tim,Boothe
Gerald,Butters
Joan,Coplin
Erica,Crumpet
Nancy,Dare
Matthew,Genting
John,Hunt


## 2.4 Produce a list of all members, along with their recommender

In [ ]:
# How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).
runSelect("select w.firstname as memfname, w.surname as memsname, m.firstname as recfname, m.surname as recsname from cd.members w LEFT OUTER JOIN cd.members m ON m.memid = w.recommendedby order by memsname, memfname")

memfname,memsname,recfname,recsname
Florence,Bader,Ponder,Stibbons
Anne,Baker,Ponder,Stibbons
Timothy,Baker,Jemima,Farrell
Tim,Boothe,Tim,Rownam
Gerald,Butters,Darren,Smith
Joan,Coplin,Timothy,Baker
Erica,Crumpet,Tracy,Smith
Nancy,Dare,Janice,Joplette
David,Farrell,None,None
Jemima,Farrell,None,None


## 2.5 Produce a list of all members who have used a tennis court

In [ ]:
# How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.
runSelect("select distinct w.firstname ||' '||  w.surname as member, m.name as facility from cd.members w, cd.bookings d, cd.facilities m where w.memid = d.memid and d.facid = m.facid and m.name in ('Tennis Court 2', 'Tennis Court 1') order by member, facility")

member,facility
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Burton Tracy,Tennis Court 1
Burton Tracy,Tennis Court 2
Charles Owen,Tennis Court 1
Charles Owen,Tennis Court 2
Darren Smith,Tennis Court 2
David Farrell,Tennis Court 1
David Farrell,Tennis Court 2
David Jones,Tennis Court 1


## 2.6 Produce a list of costly bookings

In [ ]:
# How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.
runSelect("select w.firstname ||' '|| w.surname as member, m.name as facility, case when w.memid = 0 then d.slots*m.guestcost else d.slots*m.membercost end as cost from cd.members w, cd.bookings d, cd.facilities m where w.memid = d.memid and d.facid = m.facid and d.starttime >= '2012-09-14' and d.starttime < '2012-09-15' and ((w.memid = 0 and d.slots*m.guestcost > 30) or (w.memid != 0 and d.slots*m.membercost > 30)) order by cost desc")

member,facility,cost
GUEST GUEST,Massage Room 2,320
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Tennis Court 2,150
Jemima Farrell,Massage Room 1,140
GUEST GUEST,Tennis Court 1,75
GUEST GUEST,Tennis Court 2,75
GUEST GUEST,Tennis Court 1,75
Matthew Genting,Massage Room 1,70


## 2.7 Produce a list of all members, along with their recommender, using no joins.

In [ ]:
# How can you output a list of all members, including the individual who recommended them (if any), without using any joins? Ensure that there are no duplicates in the list, and that each firstname + surname pairing is formatted as a column and ordered.
runSelect("select w.firstname ||' '|| w.surname as member, (select m.firstname ||' '|| m.surname as recmmonder from cd.members m where m.memid = w.recommendedby) from cd.members w order by member")

member,recmmonder
Anna Mackenzie,Darren Smith
Anne Baker,Ponder Stibbons
Burton Tracy,None
Charles Owen,Darren Smith
Darren Smith,None
Darren Smith,None
David Farrell,None
David Jones,Janice Joplette
David Pinker,Jemima Farrell
Douglas Jones,David Jones


## 2.8 Produce a list of costly bookings, using a subquery


In [ ]:
# How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user is always ID 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost.
runSelect("select member, facility, cost from (select w.firstname ||' '|| w.surname as member, m.name as facility, case when w.memid = 0 then d.slots*guestcost else d.slots*membercost end as cost from cd.members w, cd.bookings d, cd.facilities m where w.memid = d.memid and d.facid = m.facid and d.starttime >= '2012-09-14' and d.starttime < '2012-09-15') as bookings where cost > 30 order by cost desc")

member,facility,cost
GUEST GUEST,Massage Room 2,320
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Massage Room 1,160
GUEST GUEST,Tennis Court 2,150
Jemima Farrell,Massage Room 1,140
GUEST GUEST,Tennis Court 1,75
GUEST GUEST,Tennis Court 2,75
GUEST GUEST,Tennis Court 1,75
Matthew Genting,Massage Room 1,70


# Modifying Data

## 3.1 Insert some data into a table

In [ ]:
# The club is adding a new facility - a spa. We need to add it into the facilities table. Use the following values:

#facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.


In [ ]:
runCmd("insert into cd.facilities (facid, name, membercost, guestcost, initialoutlay, monthlymaintenance) VALUES\
(9, 'Spa', 20, 30, 100000, 800)");

Command executed


In [ ]:
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15
9,Spa,20,30,100000,800


## 3.2 Insert multiple rows of data into a table

In [ ]:
# In the previous exercise, you learned how to add a facility. Now you're going to add multiple facilities in one command. Use the following values:

#facid: 9, Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.
#facid: 10, Name: 'Squash Court 2', membercost: 3.5, guestcost: 17.5, initialoutlay: 5000, monthlymaintenance: 80.


In [ ]:
runCmd("insert into cd.facilities(facid, name, membercost, guestcost, initialoutlay, monthlymaintenance) VALUES\
 (10, 'Squash Court 2', 3.5, 17.5, 5000, 80)");

Command executed


In [ ]:
runSelect("select * from cd.facilities")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5,25,10000,200
1,Tennis Court 2,5,25,8000,200
2,Badminton Court,0,15.5,4000,50
3,Table Tennis,0,5,320,10
4,Massage Room 1,35,80,4000,3000
5,Massage Room 2,35,80,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0,5,450,15
8,Pool Table,0,5,400,15
9,Spa,20,30,100000,800


## 3.3 Insert calculated data into a table

In [ ]:
# Let's try adding the spa to the facilities table again. This time, though, we want to automatically generate the value for the next facid, rather than specifying it as a constant. Use the following values for everything else:

#Name: 'Spa', membercost: 20, guestcost: 30, initialoutlay: 100000, monthlymaintenance: 800.

In [ ]:
runCmd("insert into cd.facilities (facid, name, membercost, guestcost, initialoutlay, monthymaintenance) select (select max(facid) from cd.facilities)+1, 'Spa', 20, 30, 100000, 800");

Command failed


## 3.5 Update some existing data


In [ ]:
# We made a mistake when entering the data for the second tennis court. The initial outlay was 10000 rather than 8000: you need to alter the data to fix the error.

In [ ]:
runCmd("update cd.facilities set initialoutlay = 10000 where facid = 1")

Command executed


## 3.6 Update multiple rows and columns at the same time

In [ ]:
# We want to increase the price of the tennis courts for both members and guests. Update the costs to be 6 for members, and 30 for guests.

In [ ]:
runCmd("update cd.facilities set membercost = 6, guestcost = 30 where facid in (0,1)")

Command executed


## 3.7 Update a row based on the contents of another row

In [ ]:
# We want to alter the price of the second tennis court so that it costs 10% more than the first one. Try to do this without using constant values for the prices, so that we can reuse the statement if we want to.

In [ ]:
runCmd("update cd.facilities set membercost = (select membercost * 1.1 from cd.facilities where facid = 0), guestcost = (select guestcost * 1.1 from cd.facilities where facid = 0) where cd.facilities.facid = 1")

Command executed


## 3.8 Delete all bookings


In [ ]:
# As part of a clearout of our database, we want to delete all bookings from the cd.bookings table. How can we accomplish this?

In [ ]:
runCmd("delete from cd.bookings")

Command executed


## 3.9 Delete a member from the cd.members table


In [ ]:
# We want to remove member 37, who has never made a booking, from our database. How can we achieve that?

In [ ]:
runCmd("delete from cd.members where memid = 37")

Command executed


## 3.10 Delete based on a subquery


In [ ]:
# In our previous exercises, we deleted a specific member who had never made a booking. How can we make that more general, to delete all members who have never made a booking?

In [ ]:
runCmd("delete from cd.members where memid not in (select memid from cd.bookings)")

Command executed


# Aggregate

## 4.1 Count the number of facilities


In [ ]:
# For our first foray into aggregates, we're going to stick to something simple. We want to know how many facilities exist - simply produce a total count.

In [ ]:
runSelect("select count(name) as count from cd.facilities")

count
11


## 4.2 Count the number of expensive facilities


In [ ]:
# Produce a count of the number of facilities that have a cost to guests of 10 or more.

In [ ]:
runSelect("select count(name) as count from cd.facilities where guestcost >= 10")

count
8


## 4.3 Count the number of recommendations each member makes.

In [ ]:
# Produce a count of the number of recommendations each member has made. Order by member ID.

In [ ]:
runSelect("select recommendedby, count(*) as count from cd.members where recommendedby is not null group by recommendedby order by recommendedby")

no records found


## 4.4 List the total slots booked per facility

In [ ]:
# Produce a list of the total number of slots booked per facility. For now, just produce an output table consisting of facility id and slots, sorted by facility id.

In [ ]:
runSelect("select facid, sum(slots) as Total_Slots from cd.bookings group by facid order by facid")

no records found


## 4.5 List the total slots booked per facility in a given month

In [ ]:
# Produce a list of the total number of slots booked per facility in the month of September 2012. Produce an output table consisting of facility id and slots, sorted by the number of slots.

In [ ]:
runSelect("select facid, sum(slots) as Total_Slots from cd.bookings where starttime >= '2012-09-01' and starttime < '2012-09-30' group by facid order by sum(slots)")

no records found


## 4.6 List the total slots booked per facility per month

In [ ]:
# Produce a list of the total number of slots booked per facility per month in the year of 2012. Produce an output table consisting of facility id and slots, sorted by the id and month.

In [ ]:
runSelect("select facid, extract(month from starttime) as month, sum(slots) as Total_Slots from cd.bookings where extract(year from starttime) = 2012 group by facid, month order by facid, month")

no records found


## 4.7 Find the count of members who have made at least one booking

In [ ]:
# Find the total number of members (including guests) who have made at least one booking.

In [ ]:
runSelect("select count(*) from (select distinct memid from cd.bookings) as d")

count
0


## 4.8 List facilities with more than 1000 slots booked


In [ ]:
# Produce a list of facilities with more than 1000 slots booked. Produce an output table consisting of facility id and slots, sorted by facility id.

In [ ]:
runSelect("select facid, sum(slots) as Total_Slots from cd.bookings group by facid having sum(slots) > 1000 order by facid")

no records found


## 4.9 Find the total revenue of each facility

In [ ]:
# Produce a list of facilities along with their total revenue. The output table should consist of facility name and revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [ ]:
runSelect("select m.name, sum(slots * case when memid = 0 then m.guestcost else m.membercost end) as revenue from cd.bookings d INNER JOIN cd.facilities m ON d.facid = m.facid group by m.name order by revenue")

no records found


## 4.10 Find facilities with a total revenue less than 1000
    

In [ ]:
# Produce a list of facilities with a total revenue less than 1000. Produce an output table consisting of facility name and revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [ ]:
runSelect("select name, revenue from (select m.name, sum(case when memid = 0 then m.guestcost else m.membercost end) as revenue from cd.bookings d INNER JOIN cd.facilities m ON d.facid = m.facid group by m.name) as total where revenue < 1000 order by revenue")

no records found


## 4.11 Output the facility id that has the highest number of slots booked

In [ ]:
# Output the facility id that has the highest number of slots booked. For bonus points, try a version without a LIMIT clause. This version will probably look messy!

In [ ]:
runSelect("select facid, max(totalslots) from (select facid, sum(slots) as totalslots from cd.bookings group by facid) as tsl group by facid")

no records found


## 4.12 List the total slots booked per facility per month, part 2

In [ ]:
# Produce a list of the total number of slots booked per facility per month in the year of 2012. In this version, include output rows containing totals for all months per facility, and a total for all months for all facilities. The output table should consist of facility id, month and slots, sorted by the id and month. When calculating the aggregated values for all months and all facids, return null values in the month and facid columns.

In [ ]:
runSelect("select facid, extract(month from starttime) as month, sum(slots) as slots from cd.bookings where starttime >= '2012-01-01' and starttime < '2012-01-01' group by rollup(facid, month) order by facid, month")

facid,month,slots
None,None,None


## 4.13 List the total hours booked per named facility

In [ ]:
# Produce a list of the total number of hours booked per facility, remembering that a slot lasts half an hour. The output table should consist of the facility id, name, and hours booked, sorted by facility id. Try formatting the hours to two decimal places.

In [ ]:
runSelect("select m.facid, m.name, trim(to_char(sum(d.slots)/2.0, '9999999999999999D99')) as Total_Hours from cd.bookings d INNER JOIN cd.facilities m ON m.facid = d.facid group by m.facid, m.name order by m.facid")

no records found


## 4.14 List each member's first booking after September 1st 2012

In [ ]:
# Produce a list of each member name, id, and their first booking after September 1st 2012. Order by member ID.

In [ ]:
runSelect("select n.surname, n.firstname, n.memid, min(d.starttime) as starttime from cd.bookings d, cd.members n where n.memid = d.memid and starttime >= '2012-09-01' group by n.surname, n.firstname, n.memid order by n.memid")

no records found


## 4.15 Produce a list of member names, with each row containing the total member count

In [ ]:
# Produce a list of member names, with each row containing the total member count. Order by join date, and include guest members.

In [ ]:
runSelect("select count(*) over(), firstname, surname from cd.members order by joindate")

no records found


## 4.16 Produce a numbered list of members

In [ ]:
# Produce a monotonically increasing numbered list of members (including guests), ordered by their date of joining. Remember that member IDs are not guaranteed to be sequential.

In [ ]:
runSelect("select row_number() over(order by joindate), firstname, surname from cd.members order by joindate")

no records found


## 4.17 Output the facility id that has the highest number of slots booked, again

In [ ]:
# Output the facility id that has the highest number of slots booked. Ensure that in the event of a tie, all tieing results get output.

In [ ]:
runSelect("select facid, total from(select facid, sum(slots) as total, rank() over (order by sum(slots) desc) rank from cd.bookings group by facid) as ranked where rank = 1")

no records found


## 4.18 Rank members by (rounded) hours used

In [ ]:
# Produce a list of members (including guests), along with the number of hours they've booked in facilities, rounded to the nearest ten hours. Rank them by this rounded figure, producing output of first name, surname, rounded hours, rank. Sort by rank, surname, and first name.

In [ ]:
runSelect("select firstname, surname, ((sum(d.slots)+10)/20)*10 as hours, rank() over(order by ((sum(d.slots)+10)/20)*10 desc) as rank from cd.bookings d, cd.members n where d.memid = n.memid group by n.memid order by rank, surname, firstname")

no records found


## 4.19 Find the top three revenue generating facilities

In [ ]:
# Produce a list of the top three revenue generating facilities (including ties). Output facility name and rank, sorted by rank and facility name.

In [ ]:
runSelect("select name, rank from (select m.name as name, rank() over(order by sum(case when memid = 0 then slots*m.guestcost else slots*m.membercost end)desc) as rank from cd.bookings d, cd.facilities m where d.facid = m.facid group by m.name) as tsl where rank <= 3 order by rank")

no records found


## 4.20 Classify facilities by value

In [ ]:
# Classify facilities into equally sized groups of high, average, and low based on their revenue. Order by classification and facility name.

In [ ]:
runSelect("select name, case when class=1 then 'high' when class=2 then 'average' else 'low' end revenue from (select m.name as name, ntile(3) over (order by sum(case when memid = 0 then slots * m.guestcost else slots * membercost end) desc) as class from cd.bookings d, cd.facilities m where d.facid = m.facid group by m.name) as tslq order by class, name")

no records found


## 4.21 Calculate the payback time for each facility

In [ ]:
# Based on the 3 complete months of data so far, calculate the amount of time each facility will take to repay its cost of ownership. Remember to take into account ongoing monthly maintenance. Output facility name and payback time in months, order by facility name. Don't worry about differences in month lengths, we're only looking for a rough value here!

In [ ]:
runSelect("select m.name as name, m.initialoutlay/((sum(case when memid = 0 then slots*m.guestcost else slots*membercost end)/ 3) - m.monthlymaintenance) as months from cd.bookings d, cd.facilities m where d.facid = m.facid group by m.facid order by name")

no records found


## 4.22 Calculate a rolling average of total revenue

In [ ]:
# For each day in August 2012, calculate a rolling average of total revenue over the previous 15 days. Output should contain date and revenue columns, sorted by the date. Remember to account for the possibility of a day having zero revenue. This one's a bit tough, so don't be afraid to check out the hint!

In [ ]:
runSelect("select dategen.date, (select sum(case when memid = 0 then slots*m.guestcost else slots*membercost end) as rev from cd.bookings d, cd.facilities m where d.facid = m.facid and d.starttime > dategen.date - interval '14 days' and d.starttime < dategen.date + interval '1 day')/15 as revenue from (select cast(generate_series(timestamp '2012-08-01', '2012-08-31','1 day') as date) as date) as dategen order by dategen.date")

date,revenue
2012-08-01,None
2012-08-02,None
2012-08-03,None
2012-08-04,None
2012-08-05,None
2012-08-06,None
2012-08-07,None
2012-08-08,None
2012-08-09,None
2012-08-10,None


# Date

## 4.1 Produce a timestamp for 1 a.m. on the 31st of August 2012


In [ ]:
# Produce a timestamp for 1 a.m. on the 31st of August 2012.
runSelect("select timestamp '2018-08-31 01:00:00'")

timestamp
2018-08-31 01:00:00


## 4.2 Subtract timestamps from each other

In [ ]:
# Find the result of subtracting the timestamp '2012-07-30 01:00:00' from the timestamp '2012-08-31 01:00:00'
runSelect("select timestamp '2012-08-31 01:00:00' - timestamp'2012-07-30 01:00:00' as interval")

interval
32 days 00:00:00


## 4.4 Generate a list of all the dates in October 2012


In [ ]:
# Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
runSelect("select generate_series(timestamp '2012-10-01', timestamp '2012-10-31', interval '1 day') as ts")

ts
2012-10-01 00:00:00
2012-10-02 00:00:00
2012-10-03 00:00:00
2012-10-04 00:00:00
2012-10-05 00:00:00
2012-10-06 00:00:00
2012-10-07 00:00:00
2012-10-08 00:00:00
2012-10-09 00:00:00
2012-10-10 00:00:00


## 4.5 Get the day of the month from a timestamp

In [ ]:
# Get the day of the month from the timestamp '2012-08-31' as an integer.
runSelect("select extract(day from timestamp'2012-08-31') as date_part")

date_part
31


## 4.6 Work out the number of seconds between timestamps

In [ ]:
# Work out the number of seconds between the timestamps '2012-08-31 01:00:00' and '2012-09-02 00:00:00'
runSelect("select extract(epoch from(timestamp '2012-09-02 00:00:00' - timestamp  '2012-08-31 01:00:00')) as date_part")

date_part
169200.000000


## 4.7 Work out the number of days in each month of 2012

In [ ]:
# For each month of the year in 2012, output the number of days in that month. Format the output as an integer column containing the month of the year, and a second column containing an interval data type.

In [ ]:
runSelect("select extract(month from cal.month) as month, (cal.month + interval '1 month') - cal.month as length from (select generate_series(timestamp '2012-01-01', timestamp '2012-12-01', interval '1 month') as month) cal order by month")

month,length
1,31 days 00:00:00
2,29 days 00:00:00
3,31 days 00:00:00
4,30 days 00:00:00
5,31 days 00:00:00
6,30 days 00:00:00
7,31 days 00:00:00
8,31 days 00:00:00
9,30 days 00:00:00
10,31 days 00:00:00


## 4.8 Work out the number of days remaining in the month

In [ ]:
# For any given timestamp, work out the number of days remaining in the month. The current day should count as a whole day, regardless of the time. Use '2012-02-11 01:00:00' as an example timestamp for the purposes of making the answer. Format the output as a single interval value.

In [ ]:
runSelect("select(date_trunc('month', ts.ts1) + interval '1 month') - date_trunc('days', ts.ts1) as remaining from (select timestamp '2012-02-11 01:00:00' as ts1) ts")

remaining
19 days 00:00:00


## 4.9 Work out the end time of bookings

In [ ]:
# 59.1 Return a list of the start and end time of the last 10 bookings (ordered by the time at which they end, followed by the time at which they start) in the system.

In [ ]:
runSelect("select starttime, starttime + slots*(interval '30 minutes') as endtime from cd.bookings order by endtime desc, starttime desc limit 10")

no records found


## 4.10 Return a count of bookings for each month

In [ ]:
# Return a count of bookings for each month, sorted by month

In [ ]:
runSelect("select date_trunc('month', starttime) as month, count(*) from cd.bookings group by month order by month")

no records found


## 4.11 Work out the utilisation percentage for each facility by month

In [ ]:
#  Work out the utilisation percentage for each facility by month, sorted by name and month, rounded to 1 decimal place. Opening time is 8am, closing time is 8.30pm. You can treat every month as a full month, regardless of if there were some dates the club was not open.

In [ ]:
runSelect("select name, month, \
	round((100*slots)/ \
		cast( \
			25*(cast((month + interval '1 month') as date) \
			- cast (month as date)) as numeric),1) as utilisation \
	from  ( \
		select facs.name as name, date_trunc('month', starttime) as month, sum(slots) as slots \
			from cd.bookings bks \
			inner join cd.facilities facs \
				on bks.facid = facs.facid \
			group by facs.facid, month \
	) as inn \
order by name, month")

no records found


# String

## 5.1 Format the names of members

In [ ]:
# Output the names of all members, formatted as 'Surname, Firstname'

In [ ]:
runSelect("select surname ||', '|| firstname as name from cd.members")

no records found


## 5.2 Find facilities by a name prefix

In [ ]:
# Find all facilities whose name begins with 'Tennis'. Retrieve all columns.

In [ ]:
runSelect("select * from cd.facilities where name like '%Tennis%'")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
3,Table Tennis,0,5,320,10
0,Tennis Court 1,6,30,10000,200
1,Tennis Court 2,6.6,33.0,10000,200


## 5.3 Perform a case-insensitive search


In [ ]:
# Perform a case-insensitive search to find all facilities whose name begins with 'tennis'. Retrieve all columns.

In [ ]:
runSelect("select * from cd.facilities where upper(name) like '%TENNIS%'")

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
3,Table Tennis,0,5,320,10
0,Tennis Court 1,6,30,10000,200
1,Tennis Court 2,6.6,33.0,10000,200


## 5.4 Find telephone numbers with parentheses

In [ ]:
# You've noticed that the club's member table has telephone numbers with very inconsistent formatting. You'd like to find all the telephone numbers that contain parentheses, returning the member ID and telephone number sorted by member ID.

In [ ]:
runSelect("select memid, telephone from cd.members where telephone ~ '[()]' order by memid")

no records found


## 5.5 Pad zip codes with leading zeroes

In [ ]:
# The zip codes in our example dataset have had leading zeroes removed from them by virtue of being stored as a numeric type. Retrieve all zip codes from the members table, padding any zip codes less than 5 characters long with leading zeroes. Order by the new zip code.

In [ ]:
runSelect("select lpad(cast(zipcode as char(5)),5,'0') as zip from cd.members order by zip")

no records found


## 5.6 Count the number of members whose surname starts with each letter of the alphabet

In [ ]:
# You'd like to produce a count of how many members you have whose surname starts with each letter of the alphabet. Sort by the letter, and don't worry about printing out a letter if the count is 0.

In [ ]:
runSelect("select substring(m.surname,1,1) as letter, count(*) as count from cd.members m group by letter order by letter ")

no records found


## 5.7 Clean up telephone numbers

In [ ]:
# The telephone numbers in the database are very inconsistently formatted. You'd like to print a list of member ids and numbers that have had '-','(',')', and ' ' characters removed. Order by member id.

In [ ]:
runSelect("select memid, translate(telephone, '-() ', '') as telephone from cd.members order by memid")

no records found


# Recursive

## 6.1 Find the upward recommendation chain for member ID 27

In [ ]:
# Find the upward recommendation chain for member ID 27: that is, the member who recommended them, and the member who recommended that member, and so on. Return member ID, first name, and surname. Order by descending member id.

In [ ]:
runSelect("with recursive recommenders (recommender) as (select recommendedby from cd.members where memid = 27 union all select n.recommendedby from recommenders r INNER JOIN cd.members n on n.memid = r.recommender) select r.recommender, n.firstname, n.surname from recommenders r INNER JOIN cd.members n ON r.recommender = n.memid order by memid desc")

recommender,firstname,surname
20,Matthew,Genting
5,Gerald,Butters
1,Darren,Smith


## 6.2 Find the downward recommendation chain for member ID 1

In [ ]:
# Find the downward recommendation chain for member ID 1: that is, the members they recommended, the members those members recommended, and so on. Return member ID and name, and order by ascending member id.

In [ ]:
runSelect("with recursive recommended (memid) as (select memid from cd.members where recommendedby = 1 union all select n.memid from recommended r INNER JOIN cd.members n on n.recommendedby = r.memid) select r.memid, n.firstname, n.surname from recommended r INNER JOIN cd.members n ON r.memid = n.memid order by memid")

memid,firstname,surname
4,Janice,Joplette
5,Gerald,Butters
7,Nancy,Dare
10,Charles,Owen
11,David,Jones
14,Jack,Smith
20,Matthew,Genting
21,Anna,Mackenzie
26,Douglas,Jones
27,Henrietta,Rumney


## 6.3 Produce a CTE that can return the upward recommendation chain for any member

In [ ]:
# Produce a CTE that can return the upward recommendation chain for any member. You should be able to select recommender from recommenders where member=x. Demonstrate it by getting the chains for members 12 and 22. Results table should have member and recommender, ordered by member ascending, recommender descending.

In [ ]:
runSelect("with recursive recommenders(recommender, member) as (select recommendedby, memid from cd.members union all select n.recommendedby, r.member from recommenders r inner join cd.members n on n.memid = r.recommender) select r.member member, r.recommender, n.firstname, n.surname from recommenders r inner join cd.members n on r.recommender = n.memid where r.member = 22 or r.member = 12 order by r.member asc, r.recommender desc ")

member,recommender,firstname,surname
12,9,Ponder,Stibbons
12,6,Burton,Tracy
22,16,Timothy,Baker
22,13,Jemima,Farrell


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [ ]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-02-29 12:58:30.465662+05:30
